In [1]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import os
from supervised.automl import AutoML

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
data1 = pd.read_excel('data/FrontiersDS.xlsx', sheet_name='Wide')

In [3]:
data2 = pd.read_excel('data/FrontiersDS.xlsx', sheet_name='Long')

In [4]:
data1.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69
0,The data in this sheet was used for the formul...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Species,Strain,Samples,Ethyl Acetate,Ethanol,"Propanoic acid, ethyl ester",2-Pentanone,Decane,Methyl Isobutyl Ketone,Amylene hydrate,...,1-Dodecanol,Methyl tetradecanoate,2-Pentadecanone,"Tetradecanoic acid, ethyl ester",Hexadecanal,n-Tridecan-1-ol,1-Tetradecanol,n-Pentadecanol,1-Hexadecanol,Indole
3,SA,SA_A,SA.A_TSB_A,465374,1027715,NaN,1289650,800581,324424,73015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SA,SA_A,SA.A_TSB_B,193151,1050974,NaN,504113,294680,189630,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
headings = data1.loc[[2]].values
z = list(headings.flatten())

In [6]:
data1.columns = z

In [7]:
data1.head()

,Species,Strain,Samples,Ethyl Acetate,Ethanol,"Propanoic acid, ethyl ester",2-Pentanone,Decane,Methyl Isobutyl Ketone,Amylene hydrate,...,1-Dodecanol,Methyl tetradecanoate,2-Pentadecanone,"Tetradecanoic acid, ethyl ester",Hexadecanal,n-Tridecan-1-ol,1-Tetradecanol,n-Pentadecanol,1-Hexadecanol,Indole
0,The data in this sheet was used for the formul...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Species,Strain,Samples,Ethyl Acetate,Ethanol,"Propanoic acid, ethyl ester",2-Pentanone,Decane,Methyl Isobutyl Ketone,Amylene hydrate,...,1-Dodecanol,Methyl tetradecanoate,2-Pentadecanone,"Tetradecanoic acid, ethyl ester",Hexadecanal,n-Tridecan-1-ol,1-Tetradecanol,n-Pentadecanol,1-Hexadecanol,Indole
3,SA,SA_A,SA.A_TSB_A,465374,1027715,NaN,1289650,800581,324424,73015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SA,SA_A,SA.A_TSB_B,193151,1050974,NaN,504113,294680,189630,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
data1 = data1.drop([0,1], axis=0)

In [9]:
# forgot to remove a row
data1 = data1.drop([2], axis=0)
data1.head()

,Species,Strain,Samples,Ethyl Acetate,Ethanol,"Propanoic acid, ethyl ester",2-Pentanone,Decane,Methyl Isobutyl Ketone,Amylene hydrate,...,1-Dodecanol,Methyl tetradecanoate,2-Pentadecanone,"Tetradecanoic acid, ethyl ester",Hexadecanal,n-Tridecan-1-ol,1-Tetradecanol,n-Pentadecanol,1-Hexadecanol,Indole
3,SA,SA_A,SA.A_TSB_A,465374,1027715,NaN,1289650,800581,324424,73015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SA,SA_A,SA.A_TSB_B,193151,1050974,NaN,504113,294680,189630,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,SA,SA_A,SA.A_TSB_C,403286,1850391,NaN,1169501,15,228163,73558,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,SA,SA_A,SA.A_TSB_D,129833,5140770,NaN,1926072,124282,0,188367,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,SA,SA_A,SA.A_TSB_E,117105,3422557,NaN,246751,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
data2.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Compounds,Chemical_class,NaN,NaN,SA.A_TSB_A,SA.A_TSB_B,SA.A_TSB_C,SA.A_TSB_D,SA.A_TSB_E,SA.B_TSB_A,...,EC.A_TSB_A,EC.A_TSB_B,EC.A_TSB_C,EC.A_TSB_D,EC.A_TSB_E,EC.B_TSB_A,EC.B_TSB_B,EC.B_TSB_C,EC.B_TSB_D,EC.B_TSB_E
2,Ethyl Acetate,Ester,888±8 (234),43.0,465374,193151,403286,129833,117105,316764,...,887825,670603,576884,636851,NaN,119313,128397,309004,548442,339404
3,Ethanol,Alcohol,932±8 (,45.0,1027715,1050974,1850391,5140770,3422557,914667,...,7585417,7461659,2372532,5114072,7870420,1364553,1451138,5650813,3588158,3962927
4,"Propanoic acid, ethyl ester",Fatty acid ethyl ester,953±7 (87),57.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [11]:
headings2 = data2.loc[[1]].values
w = list(headings2.flatten())
data2.columns = w
data2.head()

,Compounds,Chemical_class,NaN,NaN,SA.A_TSB_A,SA.A_TSB_B,SA.A_TSB_C,SA.A_TSB_D,SA.A_TSB_E,SA.B_TSB_A,...,EC.A_TSB_A,EC.A_TSB_B,EC.A_TSB_C,EC.A_TSB_D,EC.A_TSB_E,EC.B_TSB_A,EC.B_TSB_B,EC.B_TSB_C,EC.B_TSB_D,EC.B_TSB_E
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Compounds,Chemical_class,NaN,NaN,SA.A_TSB_A,SA.A_TSB_B,SA.A_TSB_C,SA.A_TSB_D,SA.A_TSB_E,SA.B_TSB_A,...,EC.A_TSB_A,EC.A_TSB_B,EC.A_TSB_C,EC.A_TSB_D,EC.A_TSB_E,EC.B_TSB_A,EC.B_TSB_B,EC.B_TSB_C,EC.B_TSB_D,EC.B_TSB_E
2,Ethyl Acetate,Ester,888±8 (234),43.0,465374,193151,403286,129833,117105,316764,...,887825,670603,576884,636851,NaN,119313,128397,309004,548442,339404
3,Ethanol,Alcohol,932±8 (,45.0,1027715,1050974,1850391,5140770,3422557,914667,...,7585417,7461659,2372532,5114072,7870420,1364553,1451138,5650813,3588158,3962927
4,"Propanoic acid, ethyl ester",Fatty acid ethyl ester,953±7 (87),57.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [12]:
data2 = data2.drop([0], axis=0)
data2.head()

,Compounds,Chemical_class,NaN,NaN,SA.A_TSB_A,SA.A_TSB_B,SA.A_TSB_C,SA.A_TSB_D,SA.A_TSB_E,SA.B_TSB_A,...,EC.A_TSB_A,EC.A_TSB_B,EC.A_TSB_C,EC.A_TSB_D,EC.A_TSB_E,EC.B_TSB_A,EC.B_TSB_B,EC.B_TSB_C,EC.B_TSB_D,EC.B_TSB_E
1,Compounds,Chemical_class,NaN,NaN,SA.A_TSB_A,SA.A_TSB_B,SA.A_TSB_C,SA.A_TSB_D,SA.A_TSB_E,SA.B_TSB_A,...,EC.A_TSB_A,EC.A_TSB_B,EC.A_TSB_C,EC.A_TSB_D,EC.A_TSB_E,EC.B_TSB_A,EC.B_TSB_B,EC.B_TSB_C,EC.B_TSB_D,EC.B_TSB_E
2,Ethyl Acetate,Ester,888±8 (234),43.0,465374,193151,403286,129833,117105,316764,...,887825,670603,576884,636851,NaN,119313,128397,309004,548442,339404
3,Ethanol,Alcohol,932±8 (,45.0,1027715,1050974,1850391,5140770,3422557,914667,...,7585417,7461659,2372532,5114072,7870420,1364553,1451138,5650813,3588158,3962927
4,"Propanoic acid, ethyl ester",Fatty acid ethyl ester,953±7 (87),57.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
5,2-Pentanone,Ketone,981±11 (141),43.0,1289650,504113,1169501,1926072,246751,560337,...,673781,592162,468142,559183,1225829,169495,359499,875905,637563,2988409


In [13]:
data2 = data2.drop([1], axis=0)
data2.head()

,Compounds,Chemical_class,NaN,NaN,SA.A_TSB_A,SA.A_TSB_B,SA.A_TSB_C,SA.A_TSB_D,SA.A_TSB_E,SA.B_TSB_A,...,EC.A_TSB_A,EC.A_TSB_B,EC.A_TSB_C,EC.A_TSB_D,EC.A_TSB_E,EC.B_TSB_A,EC.B_TSB_B,EC.B_TSB_C,EC.B_TSB_D,EC.B_TSB_E
2,Ethyl Acetate,Ester,888±8 (234),43.0,465374,193151,403286,129833,117105,316764,...,887825,670603,576884,636851,NaN,119313,128397,309004,548442,339404
3,Ethanol,Alcohol,932±8 (,45.0,1027715,1050974,1850391,5140770,3422557,914667,...,7585417,7461659,2372532,5114072,7870420,1364553,1451138,5650813,3588158,3962927
4,"Propanoic acid, ethyl ester",Fatty acid ethyl ester,953±7 (87),57.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
5,2-Pentanone,Ketone,981±11 (141),43.0,1289650,504113,1169501,1926072,246751,560337,...,673781,592162,468142,559183,1225829,169495,359499,875905,637563,2988409
6,Decane,Alkane,NaN,43.0,800581,294680,15,124282,0,456376,...,0,0,0,0,0,0,0,0,0,348154


In [14]:
data2

,Compounds,Chemical_class,NaN,NaN,SA.A_TSB_A,SA.A_TSB_B,SA.A_TSB_C,SA.A_TSB_D,SA.A_TSB_E,SA.B_TSB_A,...,EC.A_TSB_A,EC.A_TSB_B,EC.A_TSB_C,EC.A_TSB_D,EC.A_TSB_E,EC.B_TSB_A,EC.B_TSB_B,EC.B_TSB_C,EC.B_TSB_D,EC.B_TSB_E
2,Ethyl Acetate,Ester,888±8 (234),43.0,465374,193151,403286,129833,117105,316764,...,887825,670603,576884,636851,NaN,119313,128397,309004,548442,339404
3,Ethanol,Alcohol,932±8 (,45.0,1027715,1050974,1850391,5140770,3422557,914667,...,7585417,7461659,2372532,5114072,7870420,1364553,1451138,5650813,3588158,3962927
4,"Propanoic acid, ethyl ester",Fatty acid ethyl ester,953±7 (87),57.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
5,2-Pentanone,Ketone,981±11 (141),43.0,1289650,504113,1169501,1926072,246751,560337,...,673781,592162,468142,559183,1225829,169495,359499,875905,637563,2988409
6,Decane,Alkane,NaN,43.0,800581,294680,15,124282,0,456376,...,0,0,0,0,0,0,0,0,0,348154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,n-Tridecan-1-ol,Alcohol,2074±20,55.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
65,1-Tetradecanol,Alcohol,2165±10 (27),55.0,NaN,NaN,NaN,NaN,NaN,NaN,...,851004,922978,593032,450711,843660,0,78672,84426,146349,635257
66,n-Pentadecanol,Alcohol,2254±15,83.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
67,1-Hexadecanol,Alcohol,2382±5 (49),83.0,NaN,NaN,NaN,NaN,NaN,NaN,...,850885,1183943,382201,390122,1033300,0,0,0,0,482026


In [15]:
data2 = data2.rename(columns={np.nan: 'CI'})
data2.head()

,Compounds,Chemical_class,CI,NaN,SA.A_TSB_A,SA.A_TSB_B,SA.A_TSB_C,SA.A_TSB_D,SA.A_TSB_E,SA.B_TSB_A,...,EC.A_TSB_A,EC.A_TSB_B,EC.A_TSB_C,EC.A_TSB_D,EC.A_TSB_E,EC.B_TSB_A,EC.B_TSB_B,EC.B_TSB_C,EC.B_TSB_D,EC.B_TSB_E
2,Ethyl Acetate,Ester,888±8 (234),43.0,465374,193151,403286,129833,117105,316764,...,887825,670603,576884,636851,NaN,119313,128397,309004,548442,339404
3,Ethanol,Alcohol,932±8 (,45.0,1027715,1050974,1850391,5140770,3422557,914667,...,7585417,7461659,2372532,5114072,7870420,1364553,1451138,5650813,3588158,3962927
4,"Propanoic acid, ethyl ester",Fatty acid ethyl ester,953±7 (87),57.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
5,2-Pentanone,Ketone,981±11 (141),43.0,1289650,504113,1169501,1926072,246751,560337,...,673781,592162,468142,559183,1225829,169495,359499,875905,637563,2988409
6,Decane,Alkane,NaN,43.0,800581,294680,15,124282,0,456376,...,0,0,0,0,0,0,0,0,0,348154


In [16]:
data2.columns

Index([     'Compounds', 'Chemical_class',             'CI',              nan,
           'SA.A_TSB_A',     'SA.A_TSB_B',     'SA.A_TSB_C',     'SA.A_TSB_D',
           'SA.A_TSB_E',     'SA.B_TSB_A',     'SA.B_TSB_B',     'SA.B_TSB_C',
           'SA.B_TSB_D',     'SA.B_TSB_E',     'SA.A_BHI_A',     'SA.A_BHI_B',
           'SA.A_BHI_C',     'SA.A_BHI_D',     'SA.A_BHI_E',     'SA.B_BHI_A',
           'SA.B_BHI_B',     'SA.B_BHI_C',     'SA.B_BHI_D',     'SA.B_BHI_E',
            'SA.A_LB_A',      'SA.A_LB_B',      'SA.A_LB_C',      'SA.A_LB_D',
            'SA.A_LB_E',      'SA.B_LB_A',      'SA.B_LB_B',      'SA.B_LB_C',
            'SA.B_LB_D',      'SA.B_LB_E',     'PA.A_BHI_A',     'PA.A_BHI_B',
           'PA.A_BHI_C',     'PA.A_BHI_D',     'PA.A_BHI_E',     'PA.B_BHI_A',
           'PA.B_BHI_B',     'PA.B_BHI_C',     'PA.B_BHI_D',     'PA.B_BHI_E',
            'PA.A_LB_A',      'PA.A_LB_B',      'PA.A_LB_C',      'PA.A_LB_D',
            'PA.A_LB_E',      'PA.B_LB_A',      'PA.

In [17]:
data2.columns = data2.columns.fillna('SD')
data2.head()

,Compounds,Chemical_class,CI,SD,SA.A_TSB_A,SA.A_TSB_B,SA.A_TSB_C,SA.A_TSB_D,SA.A_TSB_E,SA.B_TSB_A,...,EC.A_TSB_A,EC.A_TSB_B,EC.A_TSB_C,EC.A_TSB_D,EC.A_TSB_E,EC.B_TSB_A,EC.B_TSB_B,EC.B_TSB_C,EC.B_TSB_D,EC.B_TSB_E
2,Ethyl Acetate,Ester,888±8 (234),43.0,465374,193151,403286,129833,117105,316764,...,887825,670603,576884,636851,NaN,119313,128397,309004,548442,339404
3,Ethanol,Alcohol,932±8 (,45.0,1027715,1050974,1850391,5140770,3422557,914667,...,7585417,7461659,2372532,5114072,7870420,1364553,1451138,5650813,3588158,3962927
4,"Propanoic acid, ethyl ester",Fatty acid ethyl ester,953±7 (87),57.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
5,2-Pentanone,Ketone,981±11 (141),43.0,1289650,504113,1169501,1926072,246751,560337,...,673781,592162,468142,559183,1225829,169495,359499,875905,637563,2988409
6,Decane,Alkane,NaN,43.0,800581,294680,15,124282,0,456376,...,0,0,0,0,0,0,0,0,0,348154


In [18]:
data1.head()

,Species,Strain,Samples,Ethyl Acetate,Ethanol,"Propanoic acid, ethyl ester",2-Pentanone,Decane,Methyl Isobutyl Ketone,Amylene hydrate,...,1-Dodecanol,Methyl tetradecanoate,2-Pentadecanone,"Tetradecanoic acid, ethyl ester",Hexadecanal,n-Tridecan-1-ol,1-Tetradecanol,n-Pentadecanol,1-Hexadecanol,Indole
3,SA,SA_A,SA.A_TSB_A,465374,1027715,NaN,1289650,800581,324424,73015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SA,SA_A,SA.A_TSB_B,193151,1050974,NaN,504113,294680,189630,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,SA,SA_A,SA.A_TSB_C,403286,1850391,NaN,1169501,15,228163,73558,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,SA,SA_A,SA.A_TSB_D,129833,5140770,NaN,1926072,124282,0,188367,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,SA,SA_A,SA.A_TSB_E,117105,3422557,NaN,246751,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We see a lot of NaN values in both our dataframes so far. While it could be assumed these values are 0 it is interesting to see some 0 values are already present in the dataframes. Assuming NaN is 0 is valid however, as it means in different compounds in the case of data2 there is no readings for these samples as they are not present.

We use a simple imputer based on the average value of columns to calculate NaN values.

In [19]:
nan_cols = [i for i in data1.columns if data1[i].isnull().any()]
len(nan_cols)

58

The vast majority of columns have some NaN values. We could try follow a linear regression approach to predict these NaN values but this could be a very time consuming and costly approach performing LR on each of the 58 columns with NaN values.

In [20]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
num = data1.iloc[:, 3:]
imp.fit(num)
imp.transform(num)

array([[4.65374000e+05, 1.02771500e+06, 1.79894074e+04, ...,
        1.73037615e+05, 6.62122074e+05, 8.24690630e+07],
       [1.93151000e+05, 1.05097400e+06, 1.79894074e+04, ...,
        1.73037615e+05, 6.62122074e+05, 8.24690630e+07],
       [4.03286000e+05, 1.85039100e+06, 1.79894074e+04, ...,
        1.73037615e+05, 6.62122074e+05, 8.24690630e+07],
       ...,
       [3.09004000e+05, 5.65081300e+06, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 7.21023920e+07],
       [5.48442000e+05, 3.58815800e+06, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 8.52575460e+07],
       [3.39404000e+05, 3.96292700e+06, 0.00000000e+00, ...,
        0.00000000e+00, 4.82026000e+05, 9.71565490e+07]])

In [21]:
imputed_data1 = pd.DataFrame(imp.transform(num))
heading_d1 = data1.iloc[:, 3:].columns
imputed_data1.columns = heading_d1
imputed_data1

,Ethyl Acetate,Ethanol,"Propanoic acid, ethyl ester",2-Pentanone,Decane,Methyl Isobutyl Ketone,Amylene hydrate,"Butanoic acid, 2-methyl-, methyl ester",Isobutyl acetate,Methyl isovalerate,...,1-Dodecanol,Methyl tetradecanoate,2-Pentadecanone,"Tetradecanoic acid, ethyl ester",Hexadecanal,n-Tridecan-1-ol,1-Tetradecanol,n-Pentadecanol,1-Hexadecanol,Indole
0,465374.0,1027715.0,17989.407407,1289650.0,800581.0,324424.000000,73015.0,287247.703704,46016.203704,0.000000,...,2.098782e+06,97878.333333,85753.37037,10167.740741,94159.37037,85581.62963,586042.296296,173037.615385,662122.074074,82469063.0
1,193151.0,1050974.0,17989.407407,504113.0,294680.0,189630.000000,0.0,287247.703704,46016.203704,0.000000,...,2.098782e+06,97878.333333,85753.37037,10167.740741,94159.37037,85581.62963,586042.296296,173037.615385,662122.074074,82469063.0
2,403286.0,1850391.0,17989.407407,1169501.0,15.0,228163.000000,73558.0,287247.703704,46016.203704,0.000000,...,2.098782e+06,97878.333333,85753.37037,10167.740741,94159.37037,85581.62963,586042.296296,173037.615385,662122.074074,82469063.0
3,129833.0,5140770.0,17989.407407,1926072.0,124282.0,0.000000,188367.0,287247.703704,46016.203704,0.000000,...,2.098782e+06,97878.333333,85753.37037,10167.740741,94159.37037,85581.62963,586042.296296,173037.615385,662122.074074,82469063.0
4,117105.0,3422557.0,17989.407407,246751.0,0.0,0.000000,0.0,287247.703704,46016.203704,0.000000,...,2.098782e+06,97878.333333,85753.37037,10167.740741,94159.37037,85581.62963,586042.296296,173037.615385,662122.074074,82469063.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,119313.0,1364553.0,0.000000,169495.0,0.0,218195.561404,0.0,287247.703704,0.000000,97276.929825,...,2.896150e+05,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,60325589.0
80,128397.0,1451138.0,0.000000,359499.0,0.0,218195.561404,0.0,287247.703704,0.000000,97276.929825,...,7.499420e+05,0.000000,0.00000,0.000000,0.00000,0.00000,78672.000000,0.000000,0.000000,107880324.0
81,309004.0,5650813.0,0.000000,875905.0,0.0,218195.561404,109170.0,287247.703704,0.000000,97276.929825,...,4.793150e+05,0.000000,0.00000,0.000000,0.00000,0.00000,84426.000000,0.000000,0.000000,72102392.0
82,548442.0,3588158.0,0.000000,637563.0,0.0,218195.561404,60834.0,287247.703704,0.000000,97276.929825,...,2.750809e+06,0.000000,0.00000,0.000000,0.00000,0.00000,146349.000000,0.000000,0.000000,85257546.0


In [22]:
data2['Chemical_class'].value_counts()

Alcohol                   24
Ketone                    12
Fatty acid ethyl ester    11
Ester                      6
Acid                       4
Alkane                     2
Pyrrole                    2
Aldehyde                   2
Alkene                     1
Nitrile                    1
Indole                     1
Lactone                    1
Name: Chemical_class, dtype: int64

In [23]:
training_features1 = data1.iloc[:, : 3]
training_features1

,Species,Strain,Samples
3,SA,SA_A,SA.A_TSB_A
4,SA,SA_A,SA.A_TSB_B
5,SA,SA_A,SA.A_TSB_C
6,SA,SA_A,SA.A_TSB_D
7,SA,SA_A,SA.A_TSB_E
...,...,...,...
82,EC,EC_B,EC.B_TSB_A
83,EC,EC_B,EC.B_TSB_B
84,EC,EC_B,EC.B_TSB_C
85,EC,EC_B,EC.B_TSB_D


In [24]:
pd.value_counts(training_features1.Species, normalize = True)

SA    0.357143
PA    0.321429
EC    0.321429
Name: Species, dtype: float64

In [25]:
pd.value_counts(training_features1.Strain, normalize = True)

SA_A    0.178571
SA_B    0.178571
PA_A    0.166667
EC_A    0.166667
PA_B    0.154762
EC_B    0.154762
Name: Strain, dtype: float64

In [26]:
pd.value_counts(training_features1.iloc[:, 2], normalize = True)

PA.A_LB_B     0.011905
EC.B_LB_C     0.011905
SA.A_BHI_D    0.011905
PA.A_BHI_D    0.011905
EC.A_LB_C     0.011905
                ...   
SA.A_BHI_E    0.011905
SA.A_TSB_A    0.011905
SA.A_TSB_E    0.011905
EC.B_BHI_A    0.011905
SA.B_BHI_C    0.011905
Name: Samples , Length: 84, dtype: float64

In [27]:
data1.shape

(84, 70)

In [28]:
y_pred = ['SA'] * len(data1)
y_pred = pd.DataFrame(data = y_pred,
                      index = data1.index.values,
                      columns = ['Species'])
y_pred.head()

,Species
3,SA
4,SA
5,SA
6,SA
7,SA


In [29]:
numeric_features = data1.iloc[:, 3:]
numeric_features = pd.DataFrame(numeric_features)

In [30]:
category = data1.iloc[:, :3]

In [31]:
X_train, X_test, y_train, y_test = train_test_split(category, imputed_data1,test_size=0.15)

In [32]:
automl = AutoML(algorithms=["Decision Tree", "Linear", "Nearest Neighbors", "Baseline","Random Forest"], total_time_limit=5*60)
automl.fit(X_train, y_train)

AutoML directory: AutoML_1
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Decision Tree', 'Linear', 'Nearest Neighbors', 'Baseline', 'Random Forest']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 3 models
1_Baseline rmse 305631.809184 trained in 0.46 seconds
2_DecisionTree rmse 305995.620577 trained in 15.55 seconds
3_Linear rmse 340369.821574 trained in 8.81 seconds
* Step default_algorithms will try to check up to 2 models
4_Default_RandomForest rmse 274957.54553 trained in 3.11 seconds


X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
X does not have valid feature names, but KNeighborsRegressor was fitted with feature names


Exception while producing SHAP explanations. 'NoneType' object has no attribute 'shap_values'
Continuing ...
5_Default_NearestNeighbors rmse 346899.535925 trained in 0.79 seconds
* Step ensemble will try to check up to 1 model
Ensemble rmse 274957.54553 trained in 0.59 seconds
AutoML fit time: 41.12 seconds
AutoML best model: 4_Default_RandomForest


AutoML(algorithms=['Decision Tree', 'Linear', 'Nearest Neighbors', 'Baseline',
                   'Random Forest'],
       total_time_limit=300)

In [33]:
y_predicted = automl.predict(X_test)
y_predicted
#print(pd.DataFrame({"Predicted": y_predicted[], "Species": np.array(y_test)}))

array([336251.53095094, 225243.62519093, 225243.62519093, 111161.02300754,
       111161.02300754, 111161.02300754, 111161.02300754, 111161.02300754,
       111161.02300754, 336251.53095094, 336251.53095094, 111161.02300754,
       111161.02300754])

In [34]:
X_train25, X_test25, y_train25, y_test25 = train_test_split(category, imputed_data1,test_size=0.25)

In [35]:
automl25 = AutoML(algorithms=["Decision Tree", "Linear", "Nearest Neighbors", "Baseline","Random Forest"], total_time_limit=5*60)
automl25.fit(X_train25, y_train25)

AutoML directory: AutoML_2
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Decision Tree', 'Linear', 'Nearest Neighbors', 'Baseline', 'Random Forest']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 3 models
1_Baseline rmse 246856.95428 trained in 0.7 seconds
2_DecisionTree rmse 247754.84449 trained in 5.89 seconds
3_Linear rmse 263535.381601 trained in 1.96 seconds
* Step default_algorithms will try to check up to 2 models
4_Default_RandomForest rmse 239064.915964 trained in 3.17 seconds


X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
X does not have valid feature names, but KNeighborsRegressor was fitted with feature names


Exception while producing SHAP explanations. 'NoneType' object has no attribute 'shap_values'
Continuing ...
5_Default_NearestNeighbors rmse 264147.803008 trained in 1.16 seconds
* Step ensemble will try to check up to 1 model
Ensemble rmse 238931.388536 trained in 0.54 seconds
AutoML fit time: 25.7 seconds
AutoML best model: Ensemble


AutoML(algorithms=['Decision Tree', 'Linear', 'Nearest Neighbors', 'Baseline',
                   'Random Forest'],
       total_time_limit=300)

In [36]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2)
imputer.fit_transform(num)

array([[  465374. ,  1027715. ,        0. , ...,        0. ,        0. ,
        84102956.5],
       [  193151. ,  1050974. ,        0. , ...,   694807.5,   738730. ,
        92931159.5],
       [  403286. ,  1850391. ,        0. , ...,        0. ,        0. ,
        84102956.5],
       ...,
       [  309004. ,  5650813. ,        0. , ...,        0. ,        0. ,
        72102392. ],
       [  548442. ,  3588158. ,        0. , ...,        0. ,        0. ,
        85257546. ],
       [  339404. ,  3962927. ,        0. , ...,        0. ,   482026. ,
        97156549. ]])

In [37]:
knn_imputed = pd.DataFrame(imputer.fit_transform(num))
heading_d1 = data1.iloc[:, 3:].columns
knn_imputed.columns = heading_d1
knn_imputed

,Ethyl Acetate,Ethanol,"Propanoic acid, ethyl ester",2-Pentanone,Decane,Methyl Isobutyl Ketone,Amylene hydrate,"Butanoic acid, 2-methyl-, methyl ester",Isobutyl acetate,Methyl isovalerate,...,1-Dodecanol,Methyl tetradecanoate,2-Pentadecanone,"Tetradecanoic acid, ethyl ester",Hexadecanal,n-Tridecan-1-ol,1-Tetradecanol,n-Pentadecanol,1-Hexadecanol,Indole
0,465374.0,1027715.0,0.0,1289650.0,800581.0,324424.0,73015.0,1045093.0,74647.0,0.0,...,519778.5,0.0,0.0,0.0,0.0,0.0,39336.0,0.0,0.0,84102956.5
1,193151.0,1050974.0,0.0,504113.0,294680.0,189630.0,0.0,1268001.5,0.0,0.0,...,711040.5,0.0,45529.0,0.0,142667.5,70604.5,263691.5,694807.5,738730.0,92931159.5
2,403286.0,1850391.0,0.0,1169501.0,15.0,228163.0,73558.0,1064532.0,0.0,0.0,...,519778.5,0.0,0.0,0.0,0.0,0.0,39336.0,0.0,0.0,84102956.5
3,129833.0,5140770.0,0.0,1926072.0,124282.0,0.0,188367.0,0.0,0.0,0.0,...,4325965.5,349238.0,191111.0,0.0,0.0,0.0,359841.5,0.0,241013.0,84629470.5
4,117105.0,3422557.0,0.0,246751.0,0.0,0.0,0.0,1268001.5,74647.0,0.0,...,5461712.5,349238.0,191111.0,0.0,0.0,0.0,390803.0,0.0,241013.0,91207047.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,119313.0,1364553.0,0.0,169495.0,0.0,159997.0,0.0,549406.5,0.0,0.0,...,289615.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60325589.0
80,128397.0,1451138.0,0.0,359499.0,0.0,159997.0,0.0,1268001.5,0.0,0.0,...,749942.0,0.0,0.0,0.0,0.0,0.0,78672.0,0.0,0.0,107880324.0
81,309004.0,5650813.0,0.0,875905.0,0.0,0.0,109170.0,670738.0,0.0,165645.5,...,479315.0,0.0,0.0,0.0,0.0,0.0,84426.0,0.0,0.0,72102392.0
82,548442.0,3588158.0,0.0,637563.0,0.0,0.0,60834.0,718595.0,0.0,130092.0,...,2750809.0,0.0,0.0,0.0,0.0,0.0,146349.0,0.0,0.0,85257546.0


In [38]:
X_trainknn, X_testknn, y_trainknn, y_testknn = train_test_split(category, knn_imputed,test_size=0.15)

In [39]:
automl = AutoML(algorithms=["Decision Tree", "Linear", "Nearest Neighbors", "Baseline","Random Forest"], total_time_limit=5*60)
automl.fit(X_trainknn, y_trainknn)

AutoML directory: AutoML_3
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Decision Tree', 'Linear', 'Nearest Neighbors', 'Baseline', 'Random Forest']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 3 models
1_Baseline rmse 229605.700218 trained in 0.68 seconds
2_DecisionTree rmse 491324.331872 trained in 4.9 seconds
3_Linear rmse 778164.69303 trained in 2.19 seconds
* Step default_algorithms will try to check up to 2 models
4_Default_RandomForest rmse 175504.124782 trained in 2.98 seconds


X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
X does not have valid feature names, but KNeighborsRegressor was fitted with feature names


Exception while producing SHAP explanations. 'NoneType' object has no attribute 'shap_values'
Continuing ...
5_Default_NearestNeighbors rmse 188088.298472 trained in 1.25 seconds
* Step ensemble will try to check up to 1 model
Ensemble rmse 173853.803084 trained in 0.4 seconds
AutoML fit time: 23.09 seconds
AutoML best model: Ensemble


AutoML(algorithms=['Decision Tree', 'Linear', 'Nearest Neighbors', 'Baseline',
                   'Random Forest'],
       total_time_limit=300)

In [40]:
X_trainknn25, X_testknn25, y_trainknn25, y_testknn25 = train_test_split(category, knn_imputed,test_size=0.25)

In [41]:
automl = AutoML(algorithms=["Decision Tree", "Linear", "Nearest Neighbors", "Baseline","Random Forest"], total_time_limit=5*60)
automl.fit(X_trainknn25, y_trainknn25)

AutoML directory: AutoML_4
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Decision Tree', 'Linear', 'Nearest Neighbors', 'Baseline', 'Random Forest']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 3 models
1_Baseline rmse 253097.581351 trained in 0.62 seconds
2_DecisionTree rmse 255857.381857 trained in 4.62 seconds
3_Linear rmse 264675.491844 trained in 2.51 seconds
* Step default_algorithms will try to check up to 2 models
4_Default_RandomForest rmse 242098.464173 trained in 4.54 seconds


X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
X does not have valid feature names, but KNeighborsRegressor was fitted with feature names


Exception while producing SHAP explanations. 'NoneType' object has no attribute 'shap_values'
Continuing ...
5_Default_NearestNeighbors rmse 278028.785964 trained in 1.08 seconds
* Step ensemble will try to check up to 1 model
Ensemble rmse 242098.464173 trained in 0.48 seconds
AutoML fit time: 24.88 seconds
AutoML best model: 4_Default_RandomForest


AutoML(algorithms=['Decision Tree', 'Linear', 'Nearest Neighbors', 'Baseline',
                   'Random Forest'],
       total_time_limit=300)

In [42]:
y_predicted = automl.predict(X_testknn25)
y_predicted
#print(pd.DataFrame({"Predicted": y_predictedknn, "Target": np.array(y_testknn25)}))

array([264899.43907786, 150588.94408935, 150588.94408935, 264899.43907786,
       150588.94408935, 150588.94408935, 150588.94408935, 150588.94408935,
       150588.94408935, 150588.94408935, 150588.94408935, 150588.94408935,
       150588.94408935, 150588.94408935, 279258.64638683, 264899.43907786,
       150588.94408935, 150588.94408935, 150588.94408935, 150588.94408935,
       279258.64638683])

In [43]:
data1.head()

,Species,Strain,Samples,Ethyl Acetate,Ethanol,"Propanoic acid, ethyl ester",2-Pentanone,Decane,Methyl Isobutyl Ketone,Amylene hydrate,...,1-Dodecanol,Methyl tetradecanoate,2-Pentadecanone,"Tetradecanoic acid, ethyl ester",Hexadecanal,n-Tridecan-1-ol,1-Tetradecanol,n-Pentadecanol,1-Hexadecanol,Indole
3,SA,SA_A,SA.A_TSB_A,465374,1027715,NaN,1289650,800581,324424,73015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SA,SA_A,SA.A_TSB_B,193151,1050974,NaN,504113,294680,189630,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,SA,SA_A,SA.A_TSB_C,403286,1850391,NaN,1169501,15,228163,73558,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,SA,SA_A,SA.A_TSB_D,129833,5140770,NaN,1926072,124282,0,188367,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,SA,SA_A,SA.A_TSB_E,117105,3422557,NaN,246751,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
X1 = pd.DataFrame(data1.iloc[:, 3:])
y1 = pd.Series(data1["Strain"])

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.1)

In [46]:
automl = AutoML(algorithms=["Decision Tree", "Linear", "Random Forest", "Nearest Neighbors", "Xgboost"],
                total_time_limit=5*60)
automl.fit(X_train, y_train)

2021-10-20 15:23:34,508 supervised.preprocessing.eda ERROR There was an issue when running EDA. could not convert string to float: 'PA_B'


AutoML directory: AutoML_5
The task is multiclass_classification with evaluation metric logloss
AutoML will use algorithms: ['Decision Tree', 'Linear', 'Random Forest', 'Nearest Neighbors', 'Xgboost']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 2 models
1_DecisionTree logloss 3.225204 trained in 24.8 seconds
2_Linear logloss 1.357509 trained in 30.86 seconds
* Step default_algorithms will try to check up to 3 models
3_Default_Xgboost logloss 1.54975 trained in 27.29 seconds
4_Default_RandomForest logloss 0.912605 trained in 23.75 seconds


X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
X does not have valid feature names, but KNeighborsClassifier was fitted with feature names


Exception while producing SHAP explanations. 'NoneType' object has no attribute 'shap_values'
Continuing ...
5_Default_NearestNeighbors logloss 3.425839 trained in 5.86 seconds
* Step ensemble will try to check up to 1 model
Ensemble logloss 0.90312 trained in 0.41 seconds
AutoML fit time: 132.42 seconds
AutoML best model: Ensemble


AutoML(algorithms=['Decision Tree', 'Linear', 'Random Forest',
                   'Nearest Neighbors', 'Xgboost'],
       total_time_limit=300)

In [47]:
y_predicted = automl.predict(X_test)
y_predicted = pd.DataFrame(y_predicted, columns={'Strain'})

In [48]:
print(pd.DataFrame({"Predicted": y_predicted["Strain"], "Target": np.array(y_test)}))

  Predicted Target
0      EC_B   EC_B
1      SA_B   SA_A
2      SA_B   SA_B
3      EC_B   EC_B
4      SA_A   SA_A
5      PA_B   PA_B
6      SA_A   SA_A
7      SA_A   SA_A
8      EC_B   PA_A


In [49]:
valuesy = np.array(y_test)

In [50]:
i = 0
count = 0
while i < len(y_predicted):
    if y_predicted['Strain'][i] == valuesy[i]:
        count += 1
        i += 1
    else:
        i += 1

print(count / len(y_predicted))

0.7777777777777778


In [51]:
X_train30, X_test30, y_train30, y_test30 = train_test_split(X1, y1, test_size=0.3)

In [52]:
automl = AutoML(algorithms=["Decision Tree", "Linear", "Random Forest", "Nearest Neighbors", "Xgboost"],
                total_time_limit=5*60)
automl.fit(X_train30, y_train30)

2021-10-20 15:25:47,389 supervised.preprocessing.eda ERROR There was an issue when running EDA. could not convert string to float: 'PA_B'


AutoML directory: AutoML_6
The task is multiclass_classification with evaluation metric logloss
AutoML will use algorithms: ['Decision Tree', 'Linear', 'Random Forest', 'Nearest Neighbors', 'Xgboost']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 2 models
1_DecisionTree logloss 5.586492 trained in 24.32 seconds
2_Linear logloss 1.5819 trained in 23.41 seconds
* Step default_algorithms will try to check up to 3 models
3_Default_Xgboost logloss 1.684719 trained in 26.56 seconds
4_Default_RandomForest logloss 1.169727 trained in 27.41 seconds


X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
X does not have valid feature names, but KNeighborsClassifier was fitted with feature names


Exception while producing SHAP explanations. 'NoneType' object has no attribute 'shap_values'
Continuing ...
5_Default_NearestNeighbors logloss 3.871066 trained in 5.96 seconds
* Step ensemble will try to check up to 1 model
Ensemble logloss 0.985326 trained in 0.41 seconds
AutoML fit time: 126.88 seconds
AutoML best model: Ensemble


AutoML(algorithms=['Decision Tree', 'Linear', 'Random Forest',
                   'Nearest Neighbors', 'Xgboost'],
       total_time_limit=300)

In [53]:
y_predicted30 = automl.predict(X_test30)
y_predicted30 = pd.DataFrame(y_predicted30, columns={'Strain'})
print(pd.DataFrame({"Predicted": y_predicted30["Strain"], "Target": np.array(y_test30)}))

   Predicted Target
0       EC_B   EC_B
1       SA_A   SA_A
2       SA_B   SA_B
3       EC_B   EC_B
4       SA_A   SA_A
5       PA_A   PA_B
6       SA_A   SA_A
7       SA_A   SA_A
8       PA_A   PA_A
9       PA_B   PA_B
10      PA_A   PA_A
11      SA_A   SA_A
12      SA_A   SA_B
13      PA_A   SA_A
14      EC_A   EC_A
15      EC_B   EC_B
16      PA_B   PA_B
17      PA_A   PA_A
18      SA_A   SA_A
19      PA_B   PA_B
20      EC_A   EC_A
21      SA_A   SA_B
22      EC_B   EC_A
23      SA_A   SA_B
24      EC_B   EC_A
25      PA_A   PA_A


In [54]:
values = np.array(y_test30)

In [55]:
i = 0
count = 0
while i < len(y_predicted30):
    if y_predicted30['Strain'][i] == values[i]:
        count += 1
        i += 1
    else:
        i += 1

print(count / len(y_predicted30))

0.7307692307692307


In [56]:
X_train20, X_test20, y_train20, y_test20 = train_test_split(X1, y1, test_size=0.2)

In [57]:
automl = AutoML(algorithms=["Decision Tree", "Linear", "Random Forest", "Nearest Neighbors", "Xgboost"],
                total_time_limit=5*60)
automl.fit(X_train20, y_train20)

2021-10-20 15:27:54,776 supervised.preprocessing.eda ERROR There was an issue when running EDA. could not convert string to float: 'PA_A'


AutoML directory: AutoML_7
The task is multiclass_classification with evaluation metric logloss
AutoML will use algorithms: ['Decision Tree', 'Linear', 'Random Forest', 'Nearest Neighbors', 'Xgboost']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 2 models
1_DecisionTree logloss 7.387692 trained in 32.81 seconds
2_Linear logloss 0.763105 trained in 25.78 seconds
* Step default_algorithms will try to check up to 3 models
3_Default_Xgboost logloss 1.623978 trained in 29.34 seconds
4_Default_RandomForest logloss 1.139257 trained in 32.15 seconds


X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
X does not have valid feature names, but KNeighborsClassifier was fitted with feature names


Exception while producing SHAP explanations. 'NoneType' object has no attribute 'shap_values'
Continuing ...
5_Default_NearestNeighbors logloss 2.81669 trained in 5.92 seconds
* Step ensemble will try to check up to 1 model
Ensemble logloss 0.763105 trained in 0.38 seconds
AutoML fit time: 146.48 seconds
AutoML best model: 2_Linear


AutoML(algorithms=['Decision Tree', 'Linear', 'Random Forest',
                   'Nearest Neighbors', 'Xgboost'],
       total_time_limit=300)

In [58]:
y_predicted20 = automl.predict(X_test20)
y_predicted20 = pd.DataFrame(y_predicted20, columns={'Strain'})
print(pd.DataFrame({"Predicted": y_predicted20["Strain"], "Target": np.array(y_test20)}))

   Predicted Target
0       EC_B   EC_B
1       SA_A   SA_A
2       SA_A   SA_B
3       EC_B   EC_B
4       SA_A   SA_A
5       PA_A   PA_B
6       SA_A   SA_A
7       SA_A   SA_A
8       PA_A   PA_A
9       PA_A   PA_B
10      PA_A   PA_A
11      SA_A   SA_A
12      SA_A   SA_B
13      SA_A   SA_A
14      EC_B   EC_A
15      EC_B   EC_B
16      PA_A   PA_B


In [59]:
i = 0
count = 0
values20 = np.array(y_test20)
while i < len(y_predicted20):
    if y_predicted20['Strain'][i] == values20[i]:
        count += 1
        i += 1
    else:
        i += 1

print(count / len(y_predicted20))

0.6470588235294118


In [60]:
X_train25, X_test25, y_train25, y_test25 = train_test_split(X1, y1, test_size=0.25)

In [61]:
automl = AutoML(algorithms=["Decision Tree", "Linear", "Random Forest", "Nearest Neighbors", "Xgboost"],
                total_time_limit=5*60)
automl.fit(X_train25, y_train25)

2021-10-20 15:30:21,546 supervised.preprocessing.eda ERROR There was an issue when running EDA. could not convert string to float: 'SA_B'


AutoML directory: AutoML_8
The task is multiclass_classification with evaluation metric logloss
AutoML will use algorithms: ['Decision Tree', 'Linear', 'Random Forest', 'Nearest Neighbors', 'Xgboost']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 2 models
1_DecisionTree logloss 2.412218 trained in 29.59 seconds
2_Linear logloss 0.666864 trained in 28.95 seconds
* Step default_algorithms will try to check up to 3 models
3_Default_Xgboost logloss 1.336914 trained in 29.29 seconds
4_Default_RandomForest logloss 1.00787 trained in 25.37 seconds


X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
X does not have valid feature names, but KNeighborsClassifier was fitted with feature names


Exception while producing SHAP explanations. 'NoneType' object has no attribute 'shap_values'
Continuing ...
5_Default_NearestNeighbors logloss 1.617412 trained in 6.02 seconds
* Step ensemble will try to check up to 1 model
Ensemble logloss 0.666864 trained in 0.66 seconds
AutoML fit time: 140.67 seconds
AutoML best model: 2_Linear


AutoML(algorithms=['Decision Tree', 'Linear', 'Random Forest',
                   'Nearest Neighbors', 'Xgboost'],
       total_time_limit=300)

In [62]:
y_predicted25 = automl.predict(X_test25)
y_predicted25 = pd.DataFrame(y_predicted25, columns={'Strain'})
print(pd.DataFrame({"Predicted": y_predicted25["Strain"], "Target": np.array(y_test25)}))

   Predicted Target
0       EC_B   EC_B
1       SA_A   SA_A
2       SA_A   SA_B
3       EC_B   EC_B
4       SA_A   SA_A
5       PA_A   PA_B
6       SA_A   SA_A
7       SA_A   SA_A
8       PA_A   PA_A
9       PA_B   PA_B
10      PA_A   PA_A
11      SA_A   SA_A
12      SA_A   SA_B
13      PA_A   SA_A
14      PA_A   EC_A
15      EC_B   EC_B
16      PA_A   PA_B
17      PA_A   PA_A
18      SA_A   SA_A
19      PA_A   PA_B
20      EC_B   EC_A


In [63]:
i = 0
count = 0
values25 = np.array(y_test25)
while i < len(y_predicted25):
    if y_predicted25['Strain'][i] == values25[i]:
        count += 1
        i += 1
    else:
        i += 1

print(count / len(y_predicted25))

0.6190476190476191


In [64]:
X_train40, X_test40, y_train40, y_test40 = train_test_split(X1, y1, test_size=0.40)

In [65]:
automl = AutoML(algorithms=["Decision Tree", "Linear", "Random Forest", "Nearest Neighbors", "Xgboost"],
                total_time_limit=5*60)
automl.fit(X_train40, y_train40)

2021-10-20 15:32:42,575 supervised.preprocessing.eda ERROR There was an issue when running EDA. could not convert string to float: 'PA_A'


AutoML directory: AutoML_9
The task is multiclass_classification with evaluation metric logloss
AutoML will use algorithms: ['Decision Tree', 'Linear', 'Random Forest', 'Nearest Neighbors', 'Xgboost']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 2 models
1_DecisionTree logloss 1.854583 trained in 24.82 seconds
2_Linear logloss 0.377057 trained in 22.69 seconds
* Step default_algorithms will try to check up to 3 models
3_Default_Xgboost logloss 0.916796 trained in 24.65 seconds
4_Default_RandomForest logloss 0.76903 trained in 33.65 seconds


X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
X does not have valid feature names, but KNeighborsClassifier was fitted with feature names


Exception while producing SHAP explanations. 'NoneType' object has no attribute 'shap_values'
Continuing ...
5_Default_NearestNeighbors logloss 0.47707 trained in 6.36 seconds
* Step ensemble will try to check up to 1 model
Ensemble logloss 0.377057 trained in 0.53 seconds
AutoML fit time: 133.89 seconds
AutoML best model: 2_Linear


AutoML(algorithms=['Decision Tree', 'Linear', 'Random Forest',
                   'Nearest Neighbors', 'Xgboost'],
       total_time_limit=300)

In [66]:
y_predicted40 = automl.predict(X_test40)
y_predicted40 = pd.DataFrame(y_predicted40, columns={'Strain'})
print(pd.DataFrame({"Predicted": y_predicted40["Strain"], "Target": np.array(y_test40)}))

   Predicted Target
0       EC_B   EC_B
1       SA_A   SA_A
2       SA_B   SA_B
3       EC_B   EC_B
4       SA_A   SA_A
5       PA_A   PA_B
6       SA_A   SA_A
7       SA_B   SA_A
8       PA_A   PA_A
9       PA_B   PA_B
10      PA_A   PA_A
11      SA_B   SA_A
12      SA_A   SA_B
13      SA_B   SA_A
14      EC_A   EC_A
15      EC_B   EC_B
16      PA_B   PA_B
17      PA_B   PA_A
18      SA_A   SA_A
19      PA_B   PA_B
20      EC_A   EC_A
21      SA_B   SA_B
22      EC_B   EC_A
23      EC_B   SA_B
24      EC_B   EC_A
25      PA_A   PA_A
26      PA_B   PA_B
27      PA_B   PA_B
28      SA_B   SA_A
29      SA_B   SA_B
30      PA_B   PA_A
31      SA_A   SA_B
32      PA_A   PA_A
33      EC_A   EC_A


In [67]:
i = 0
count = 0
values40 = np.array(y_test40)
while i < len(y_predicted40):
    if y_predicted40['Strain'][i] == values40[i]:
        count += 1
        i += 1
    else:
        i += 1

print(count / len(y_predicted40))

0.6470588235294118


In [68]:
X_train50, X_test50, y_train50, y_test50 = train_test_split(X1, y1, test_size=0.50)

In [69]:
automl = AutoML(algorithms=["Decision Tree", "Linear", "Random Forest", "Nearest Neighbors", "Xgboost"],
                total_time_limit=5*60)
automl.fit(X_train50, y_train50)

2021-10-20 15:34:56,760 supervised.preprocessing.eda ERROR There was an issue when running EDA. could not convert string to float: 'EC_A'


AutoML directory: AutoML_10
The task is multiclass_classification with evaluation metric logloss
AutoML will use algorithms: ['Decision Tree', 'Linear', 'Random Forest', 'Nearest Neighbors', 'Xgboost']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 2 models
1_DecisionTree logloss 11.51293 trained in 25.5 seconds
2_Linear logloss 0.456629 trained in 22.44 seconds
* Step default_algorithms will try to check up to 3 models
3_Default_Xgboost logloss 1.421264 trained in 25.12 seconds
4_Default_RandomForest logloss 1.63988 trained in 25.8 seconds


X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
X does not have valid feature names, but KNeighborsClassifier was fitted with feature names


Exception while producing SHAP explanations. 'NoneType' object has no attribute 'shap_values'
Continuing ...
5_Default_NearestNeighbors logloss 0.788799 trained in 6.21 seconds
* Step ensemble will try to check up to 1 model
Ensemble logloss 0.398333 trained in 0.41 seconds
AutoML fit time: 126.74 seconds
AutoML best model: Ensemble


AutoML(algorithms=['Decision Tree', 'Linear', 'Random Forest',
                   'Nearest Neighbors', 'Xgboost'],
       total_time_limit=300)

In [70]:
y_predicted50 = automl.predict(X_test50)
y_predicted50 = pd.DataFrame(y_predicted50, columns={'Strain'})
print(pd.DataFrame({"Predicted": y_predicted50["Strain"], "Target": np.array(y_test50)}))

   Predicted Target
0       EC_B   EC_B
1       SA_A   SA_A
2       SA_B   SA_B
3       EC_B   EC_B
4       SA_A   SA_A
5       PA_A   PA_B
6       SA_A   SA_A
7       SA_B   SA_A
8       PA_A   PA_A
9       PA_B   PA_B
10      PA_B   PA_A
11      SA_B   SA_A
12      SA_A   SA_B
13      SA_B   SA_A
14      EC_A   EC_A
15      EC_B   EC_B
16      PA_B   PA_B
17      PA_B   PA_A
18      SA_A   SA_A
19      PA_B   PA_B
20      EC_A   EC_A
21      SA_B   SA_B
22      EC_A   EC_A
23      SA_A   SA_B
24      EC_B   EC_A
25      PA_B   PA_A
26      PA_B   PA_B
27      PA_B   PA_B
28      SA_B   SA_A
29      SA_B   SA_B
30      PA_B   PA_A
31      SA_A   SA_B
32      PA_B   PA_A
33      EC_A   EC_A
34      PA_A   PA_A
35      PA_A   PA_B
36      EC_B   EC_B
37      SA_A   SA_A
38      SA_A   SA_A
39      EC_B   EC_A
40      EC_B   EC_B
41      EC_A   EC_A


X does not have valid feature names, but KNeighborsClassifier was fitted with feature names


In [71]:
i = 0
count = 0
values50 = np.array(y_test50)
while i < len(y_predicted50):
    if y_predicted50['Strain'][i] == values50[i]:
        count += 1
        i += 1
    else:
        i += 1

print(count / len(y_predicted50))

0.6190476190476191
